In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
#import pandas.rpy.common as com


%matplotlib inline

plt.style.use('bmh')
pd.options.display.float_format = '{:,.0f}'.format
pd.set_option("display.max_columns", 40)

Deal with missing data

In [3]:
df = pd.read_csv('../data/fanfics_metadata.csv')
df

,work_id,title,rating,category,fandom,relationship,character,additional tags,language,published,status,status date,words,chapters,comments,kudos,bookmarks,hits
0,3104510,Second Chances,Teen And Up Audiences,"M/M, F/F","Star Wars - All Media Types, Star Wars Prequel...","Obi-Wan Kenobi/Anakin Skywalker, Obi-Wan Kenob...","Leia Organa, Luke Skywalker, Anakin Skywalker,...","Age Regression/De-Aging, Soul Bond, The Force,...",English,2015-01-05,Completed,2016-04-16,"121,358",42/42,803,"1,917",446,"74,424"
1,6423526,hurricane on the edge of oblivion (with nowher...,Mature,Gen,Star Wars: The Wrath of Darth Maul - Ryder Win...,"Obi-Wan Kenobi & Xanatos, Qui-Gon Jinn & Feemo...","Obi-Wan Kenobi, Xanatos (Star Wars), Qui-Gon J...","minor OC's - Freeform, at least I'm pretty sur...",English,2016-04-01,Updated,2019-12-23,"144,982",24/?,744,"1,815",380,"28,728"
2,9552773,time to change the road you're on,General Audiences,Gen,"Star Wars - All Media Types, Star Wars: The Cl...","Anakin Skywalker & Ahsoka Tano, Ahsoka Tano an...","Ahsoka Tano, Anakin Skywalker, Kanan Jarrus, E...","AU, Time Travel Fix-It, possibly more of a tim...",English,2017-02-02,Completed,2017-12-27,"37,400",8/8,234,"1,446",340,"25,348"
3,5162474,Twin Sunrise,General Audiences,Gen,"Star Wars Original Trilogy, Star Wars: Rebels,...","Luke Skywalker & Darth Vader, Darth Vader & Ap...","Anakin Skywalker | Darth Vader, Luke Skywalker...","Grey Jedi, Alternate Universe, Sith Shenanigan...",English,2015-11-07,Completed,2018-01-14,"302,486",55/55,"1,003","1,418",382,"52,527"
4,4417469,On the Edge of the Devil's Backbone,Teen And Up Audiences,"F/M, Multi","Star Wars: Rebels, Star Wars - All Media Types",Kanan Jarrus/Hera Syndulla,"Hera Syndulla, Kanan Jarrus, Sabine Wren, Gara...","Alternate Universe, Canon-Typical Violence",English,2015-07-25,Completed,2018-10-20,"361,502",30/30,675,"1,395",255,"39,386"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,17114495,Ostateczne Poswiecenie. Nieznana historia ukry...,General Audiences,Gen,"Star Wars - All Media Types, Rogue One: A Star...",NaN,"Wilhuff Tarkin, Ahsoka Tano, Wulff Yularen",NaN,Polski,2018-12-23,Completed,2018-12-22,"5,462",2/2,nan,nan,nan,41
4316,16719585,The Life of Scoundrels,General Audiences,Gen,"Star Wars Sequel Trilogy, Star Wars Rebels",NaN,"Han Solo, Chewbacca (Star Wars), Boba Fett, Ho...","disintegrations WILL become part of your rep, ...",English,2018-11-23,Completed,2018-11-23,"5,587",12/12,nan,nan,nan,61
4317,20451119,In Night,Mature,M/M,"Star Wars: Rebels, Star Wars:KANAN",Janus Kasmir/Caleb Dume,"Caleb Dume, Janus Kasmir",NaN,Ri Ben Yu,2019-08-30,Completed,2019-08-30,"5,424",1/1,nan,nan,nan,15
4318,17136161,"""I am my prayer to you""",General Audiences,F/M,Star Wars: Rebels,"Garazeb ""Zeb"" Orrelios/Original Character(s)","Garazeb ""Zeb"" Orrelios, Original Characters","Romance, Religion, Lasan, Lasat, Prayer, Pilgr...",English,2018-12-24,Completed,2018-12-24,"1,842",1/1,nan,nan,nan,46


In [4]:
# Categorical Data
df['category'].fillna('No Category Specified', inplace=True)
df['relationship'].fillna('No Relationship Specified', inplace=True)
df['character'].fillna('No Character Specified', inplace=True)
df['additional tags'].fillna('No Additional Tags Specified', inplace=True)

# Continuous data
df['words'].fillna(0, inplace=True)
df['comments'].fillna(0, inplace=True)
df['kudos'].fillna(0, inplace=True)
df['bookmarks'].fillna(0, inplace=True)
df['hits'].fillna(0, inplace=True)

In [5]:
def num_chap(chapters):
    if '/' in chapters:
        first_chapter = chapters.split('/')[0]
        return int(first_chapter)
    else:
        return int(chapters)

In [6]:
#num_chap(df['chapters'])
df['num_chap'] = df['chapters'].apply(num_chap)
df['kudo_norm'] = df['kudos']/df['num_chap']
df['comment_norm'] = df['comments']/df['num_chap']
df

,work_id,title,rating,category,fandom,relationship,character,additional tags,language,published,status,status date,words,chapters,comments,kudos,bookmarks,hits,num_chap,kudo_norm,comment_norm
0,3104510,Second Chances,Teen And Up Audiences,"M/M, F/F","Star Wars - All Media Types, Star Wars Prequel...","Obi-Wan Kenobi/Anakin Skywalker, Obi-Wan Kenob...","Leia Organa, Luke Skywalker, Anakin Skywalker,...","Age Regression/De-Aging, Soul Bond, The Force,...",English,2015-01-05,Completed,2016-04-16,"121,358",42/42,803,"1,917",446,"74,424",42,46,19
1,6423526,hurricane on the edge of oblivion (with nowher...,Mature,Gen,Star Wars: The Wrath of Darth Maul - Ryder Win...,"Obi-Wan Kenobi & Xanatos, Qui-Gon Jinn & Feemo...","Obi-Wan Kenobi, Xanatos (Star Wars), Qui-Gon J...","minor OC's - Freeform, at least I'm pretty sur...",English,2016-04-01,Updated,2019-12-23,"144,982",24/?,744,"1,815",380,"28,728",24,76,31
2,9552773,time to change the road you're on,General Audiences,Gen,"Star Wars - All Media Types, Star Wars: The Cl...","Anakin Skywalker & Ahsoka Tano, Ahsoka Tano an...","Ahsoka Tano, Anakin Skywalker, Kanan Jarrus, E...","AU, Time Travel Fix-It, possibly more of a tim...",English,2017-02-02,Completed,2017-12-27,"37,400",8/8,234,"1,446",340,"25,348",8,181,29
3,5162474,Twin Sunrise,General Audiences,Gen,"Star Wars Original Trilogy, Star Wars: Rebels,...","Luke Skywalker & Darth Vader, Darth Vader & Ap...","Anakin Skywalker | Darth Vader, Luke Skywalker...","Grey Jedi, Alternate Universe, Sith Shenanigan...",English,2015-11-07,Completed,2018-01-14,"302,486",55/55,"1,003","1,418",382,"52,527",55,26,18
4,4417469,On the Edge of the Devil's Backbone,Teen And Up Audiences,"F/M, Multi","Star Wars: Rebels, Star Wars - All Media Types",Kanan Jarrus/Hera Syndulla,"Hera Syndulla, Kanan Jarrus, Sabine Wren, Gara...","Alternate Universe, Canon-Typical Violence",English,2015-07-25,Completed,2018-10-20,"361,502",30/30,675,"1,395",255,"39,386",30,46,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,17114495,Ostateczne Poswiecenie. Nieznana historia ukry...,General Audiences,Gen,"Star Wars - All Media Types, Rogue One: A Star...",No Relationship Specified,"Wilhuff Tarkin, Ahsoka Tano, Wulff Yularen",No Additional Tags Specified,Polski,2018-12-23,Completed,2018-12-22,"5,462",2/2,0,0,0,41,2,0,0
4316,16719585,The Life of Scoundrels,General Audiences,Gen,"Star Wars Sequel Trilogy, Star Wars Rebels",No Relationship Specified,"Han Solo, Chewbacca (Star Wars), Boba Fett, Ho...","disintegrations WILL become part of your rep, ...",English,2018-11-23,Completed,2018-11-23,"5,587",12/12,0,0,0,61,12,0,0
4317,20451119,In Night,Mature,M/M,"Star Wars: Rebels, Star Wars:KANAN",Janus Kasmir/Caleb Dume,"Caleb Dume, Janus Kasmir",No Additional Tags Specified,Ri Ben Yu,2019-08-30,Completed,2019-08-30,"5,424",1/1,0,0,0,15,1,0,0
4318,17136161,"""I am my prayer to you""",General Audiences,F/M,Star Wars: Rebels,"Garazeb ""Zeb"" Orrelios/Original Character(s)","Garazeb ""Zeb"" Orrelios, Original Characters","Romance, Religion, Lasan, Lasat, Prayer, Pilgr...",English,2018-12-24,Completed,2018-12-24,"1,842",1/1,0,0,0,46,1,0,0


In [ ]:
plt.scatter(df['words'], df.kudos)
plt.title("Kudos by Word Count")
plt.xlabel("words")
plt.ylabel("kudos")
plt.show()

In [ ]:
df['log_words'] = np.log(df['words'])
plt.scatter(df['log_words'], df.kudos)
plt.title("Kudos by Word Count (log)")
plt.xlabel("words(natural log scale)")
plt.ylabel("kudos")
plt.show()


In [ ]:
df['log_kudos'] = np.log(df['kudos'])
plt.scatter(df['log_words'], df['log_kudos'])
plt.title("Kudos (log) by Word Count (log)")
plt.xlabel("words(natural log scale)")
plt.ylabel("kudos(log)")
plt.show()

In [7]:
df.describe()

,work_id,words,comments,kudos,bookmarks,hits,num_chap,kudo_norm,comment_norm
count,"4,320","4,320","4,320","4,320","4,320","4,320","4,320","4,320","4,320"
mean,"12,724,031","8,016",17,69,8,"1,124",3,46,6
std,"4,739,127","25,829",56,107,20,"2,302",7,67,9
min,"2,409,431",0,0,0,0,0,1,0,0
25%,"9,094,924",887,2,18,1,283,1,9,1
50%,"13,074,524","2,015",6,40,3,608,1,26,4
75%,"16,178,795","5,349",13,81,8,"1,258",2,56,8
max,"22,009,057","720,618","1,081","1,917",446,"74,424",111,"1,340",212


In [ ]:
pd.set_option('float_format', '{:f}'.format)
df.corr()
#df[['comments', 'comment_norm', 'kudos', 'kudo_norm', 'bookmarks', 'hits']].corr()

In [ ]:
df.cov()

In [ ]:
plt.scatter(df['kudos'], df['hits'])
plt.title("Kudos by Hit Count")
#plt.ylim(top=10000, bottom=0)
plt.xlabel("hits")
plt.ylabel("kudos")
plt.show()

# Simple Linear Regression

### Kudos to Hits

In [ ]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

df = df.sample(frac=1).reset_index(drop=True)
# Use only one feature
#hits_X = df['hits']#diabetes_X[:, np.newaxis, 2]
hits_X = df['hits'].to_numpy().reshape(-1,1)
kudos_y = df['kudos']

# Split the data into training/testing sets
hits_X_train = hits_X[:-860]
hits_X_test = hits_X[-860:]

# Split the targets into training/testing sets
kudos_y_train = kudos_y[:-860]
kudos_y_test = kudos_y[-860:]



In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(hits_X_train, kudos_y_train)

# Make predictions using the testing set
kudo_pop_y_pred = regr.predict(hits_X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(kudos_y_test, kudo_pop_y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(kudos_y_test, kudo_pop_y_pred))

# Plot outputs
plt.scatter(hits_X_test, kudos_y_test,  color='black')
plt.plot(hits_X_test, kudo_pop_y_pred, color='blue', linewidth=3)

#plt.xticks(())
#plt.yticks(())
plt.title("Kudos by Hit Count")
plt.xlabel("hits")
plt.ylabel("kudos")

plt.show()

### Bookmarks to hits

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
# Use only one feature
#hits_X = df['hits']#diabetes_X[:, np.newaxis, 2]
hits_X = df['hits'].to_numpy().reshape(-1,1)
kudos_y = df['bookmarks']

# Split the data into training/testing sets
hits_X_train = hits_X[:-860]
hits_X_test = hits_X[-860:]

# Split the targets into training/testing sets
kudos_y_train = kudos_y[:-860]
kudos_y_test = kudos_y[-860:]
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(hits_X_train, kudos_y_train)

# Make predictions using the testing set
kudo_pop_y_pred = regr.predict(hits_X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(kudos_y_test, kudo_pop_y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(kudos_y_test, kudo_pop_y_pred))

# Plot outputs
plt.scatter(hits_X_test, kudos_y_test,  color='black')
plt.plot(hits_X_test, kudo_pop_y_pred, color='blue', linewidth=3)

#plt.xticks(())
#plt.yticks(())
plt.title("Bookmarks by Hit Count")
plt.xlabel("hits")
plt.ylabel("Bookmarks")

plt.show()


### Comments to hits

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
# Use only one feature
#hits_X = df['hits']#diabetes_X[:, np.newaxis, 2]
hits_X = df['hits'].to_numpy().reshape(-1,1)
kudos_y = df['comments']

# Split the data into training/testing sets
hits_X_train = hits_X[:-860]
hits_X_test = hits_X[-860:]

# Split the targets into training/testing sets
kudos_y_train = kudos_y[:-860]
kudos_y_test = kudos_y[-860:]
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(hits_X_train, kudos_y_train)

# Make predictions using the testing set
kudo_pop_y_pred = regr.predict(hits_X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(kudos_y_test, kudo_pop_y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(kudos_y_test, kudo_pop_y_pred))

# Plot outputs
plt.scatter(hits_X_test, kudos_y_test,  color='black')
plt.plot(hits_X_test, kudo_pop_y_pred, color='blue', linewidth=3)

#plt.xticks(())
#plt.yticks(())
plt.title("Comments by Hit Count")
plt.xlabel("hits")
plt.ylabel("Comments")

plt.show()


### Sum of comments+kudos+bookmarks to hits

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
# Use only one feature
#hits_X = df['hits']#diabetes_X[:, np.newaxis, 2]
hits_X = df['hits'].to_numpy().reshape(-1,1)
kudos_y = df['bookmarks']+df['kudos']+df['comments']

# Split the data into training/testing sets
hits_X_train = hits_X[:-860]
hits_X_test = hits_X[-860:]

# Split the targets into training/testing sets
kudos_y_train = kudos_y[:-860]
kudos_y_test = kudos_y[-860:]
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(hits_X_train, kudos_y_train)

# Make predictions using the testing set
kudo_pop_y_pred = regr.predict(hits_X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(kudos_y_test, kudo_pop_y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(kudos_y_test, kudo_pop_y_pred))

# Plot outputs
plt.scatter(hits_X_test, kudos_y_test,  color='black')
plt.plot(hits_X_test, kudo_pop_y_pred, color='blue', linewidth=3)

#plt.xticks(())
#plt.yticks(())
plt.title("Bookmarks by Hit Count")
plt.xlabel("hits")
plt.ylabel("Bookmarks")

plt.show()


### Try a residual plot to see if linear assumption is a fair one

In [ ]:
import seaborn as sns

# Dataframe containing only females.
#df_females = df[df['Gender']=='Female'].sample(500)
df_limited = df.sample(500)

# Residual plot 500 fanworks.
fig = plt.figure(figsize=(10,7))
sns.residplot(df_limited.hits, df_limited.kudos, color='magenta')

# Title and labels.
plt.title('Residual plot 500 fanworks', size=24)
plt.xlabel('hits', size=18)
plt.ylabel('kudos', size=18);

# Multiple Linear Regression


### Missing Data

Some authors have opted to hide their hit numbers. These fics should be dealt with when making predictions. First lets just drop the values. Later, we can also try adding the average. And try adding a predicted value.

In [8]:
df = df[df.hits != 0]

In [17]:


df = df.sample(frac=1).reset_index(drop=True)
# Use only one feature
#hits_X = df['hits']#diabetes_X[:, np.newaxis, 2]
hits_X = df[['hits','words', 'num_chap']]
kudos_y = df['kudos']

# Split the data into training/testing sets
hits_X_train = hits_X[:-860]
hits_X_test = hits_X[-860:]

# Split the targets into training/testing sets
kudos_y_train = kudos_y[:-860]
kudos_y_test = kudos_y[-860:]
# Create linear regression object
#regr = linear_model.LinearRegression()
regr = LinearRegression()

# Train the model using the training sets
regr.fit(hits_X_train, kudos_y_train)

# Make predictions using the testing set
kudo_pop_y_pred = regr.predict(hits_X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(kudos_y_test, kudo_pop_y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(kudos_y_test, kudo_pop_y_pred))

'''
SS_Residual = sum((kudos_y_-yhat)**2)       
SS_Total = sum((y-np.mean(y))**2)     
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X.shape[1]-1)
print('Adjusted r squared: %.2f'
     % adjusted_r_squared)
'''
regr.score(hits_X_test,kudos_y_test)

Coefficients: 
 [ 3.93457442e-02 -9.91903755e-05 -6.93036809e-01]
Mean squared error: 2777.46
Coefficient of determination: 0.75


0.7518024899224645

In [18]:
#let's make a prediction for a fic with 244 hits, 5379 words, and 42 
X = np.array([[244,5379,1]])
regr.predict(X)

array([35.66095917])

### Let's find the difference between predicted and actual and normalize this to find the ones that stand out

In [26]:
#df = df.drop(columns=['kudo_norm', 'comment_norm', 'log_words', 'log_kudos', 'pred_score_scaled', 'pred_score'])
df = df.drop(columns=['pred_kudos', 'kudo_diffs', 'kudo_diffs_avg'])
df = df[df.kudos != 0]
pd.options.display.float_format = '{:,.2f}'.format
df['pred_kudos'] = regr.predict(df[['hits','words', 'num_chap']])
df['kudo_diffs'] = df['kudos'] - df['pred_kudos']
df['kudo_diffs_avg'] = df['kudo_diffs']/df['kudos']
df.sort_values(by=['kudo_diffs_avg']).tail(20)

,work_id,title,rating,category,fandom,relationship,character,additional tags,language,published,status,status date,words,chapters,comments,kudos,bookmarks,hits,num_chap,pred_kudos,kudo_diffs,kudo_diffs_avg
3294,17583677,Only the Best!,General Audiences,Gen,"Star Wars - All Media Types, Star Wars Origina...","Obi-Wan Kenobi & Hondo Ohnaka, Hondo Ohnaka & ...","Hondo Ohnaka, Obi-Wan Kenobi, Luke Skywalker, ...","Alternate Universe, Alternate Universe - Canon...",English,2019-01-28,Completed,2019-01-28,"2,990.00",1/1,54.00,296.00,57.00,"2,102.00",1,109.00,187.00,0.63
4213,7949938,went back and put up a fight,General Audiences,Gen,Star Wars: The Clone Wars (2008) - All Media T...,"Anakin Skywalker & Ahsoka Tano, Obi-Wan Kenobi...","Ahsoka Tano, Anakin Skywalker, Obi-Wan Kenobi","Alternate Universe - Time Travel, Hugs, Post-E...",English,2016-09-04,Completed,2016-09-04,"2,026.00",1/1,42.00,"1,042.00",188.00,"8,973.00",1,379.44,662.56,0.64
2756,6427147,Ghosts on Endor,General Audiences,Gen,"Star Wars - All Media Types, Star Wars Prequel...",No Relationship Specified,"Luke Skywalker, CT-7567 | Rex","Grief/Mourning, Spoilers, Twilight of the Appr...",English,2016-04-01,Completed,2016-04-01,"5,315.00",1/1,55.00,534.00,138.00,"4,238.00",1,192.81,341.19,0.64
1353,14431131,Recovery,General Audiences,Gen,"Star Wars - All Media Types, Star Wars: Rebels",Obi-Wan Kenobi & Ahsoka Tano,"Ahsoka Tano, Obi-Wan Kenobi","A few cameos of other characters, Grand Master...",English,2018-05-04,Completed,2018-05-04,"1,867.00",1/1,29.00,236.00,37.00,"1,482.00",1,84.72,151.28,0.64
280,10746336,Searching for him,General Audiences,M/M,"Star Wars - All Media Types, Star Wars: The Cl...",CC-2224 | Cody/Obi-Wan Kenobi,"CC-2224 | Cody, Obi-Wan Kenobi, CT-7567 | Rex,...","Fix-It, Cody's journey from Revenge of the Sit...",English,2017-05-04,Completed,2017-05-04,"1,771.00",1/1,25.00,470.00,56.00,"3,568.00",1,166.80,303.20,0.65
1107,17580272,The Joy of Cooking,General Audiences,Gen,"Star Wars Prequel Trilogy, Star Wars Original ...",No Relationship Specified,"Obi-Wan Kenobi, Anakin Skywalker, Leia Organa,...","Cooking, 5 Times, Prompt Fic",English,2019-01-28,Completed,2019-01-28,"1,522.00",1/1,44.00,162.00,20.00,784.00,1,57.29,104.71,0.65
1895,5691409,And All Along You Knew My Story,General Audiences,Gen,"Star Wars Original Trilogy, Star Wars: Rebels",Ahsoka Tano & Leia Organa,"Ahsoka Tano, Leia Organa, Luke Skywalker, Han ...","the skywalker family tragedy, ahsoka meets the...",English,2016-01-10,Completed,2016-01-10,"2,577.00",1/1,53.00,"1,083.00",201.00,"9,039.00",1,381.98,701.02,0.65
1790,13995180,let me tell you a ghost story,General Audiences,Gen,Star Wars: The Clone Wars (2008) - All Media T...,Anakin Skywalker & Ahsoka Tano,"Ahsoka Tano, Anakin Skywalker","Force Ghosts, Force Ghost Anakin Skywalker, Hu...",English,2018-03-17,Completed,2018-03-17,"1,036.00",1/1,42.00,527.00,82.00,"4,031.00",1,185.09,341.91,0.65
1542,7549513,The Bitter End,Not Rated,Gen,Star Wars: The Clone Wars (2008) - All Media T...,Anakin Skywalker & Ahsoka Tano,"Anakin Skywalker | Darth Vader, Ahsoka Tano",i'm constantly miserable over snips and skyguy...,English,2016-07-21,Completed,2016-07-21,"1,284.00",1/1,84.00,355.00,59.00,"2,492.00",1,124.52,230.48,0.65
1909,18600613,Of Dust & Fire,Teen And Up Audiences,No Category Specified,"Star Wars: Rebels, Star Wars: Rebellion Era - ...","Ezra Bridger & Kanan Jarrus, Ezra Bridger & Sa...","Sabine Wren, Ezra Bridger, Kanan Jarrus, Orson...","Angst, Heavy Angst, mature themes, Rated For V...",English,2019-04-25,Completed,2019-12-17,"35,871.00",28/28,53.00,286.00,3.00,"2,418.00",28,99.46,186.54,0.65


In [28]:
from sklearn import preprocessing
pd.options.display.float_format = '{:,.2f}'.format
x = df['kudo_diffs_avg'].to_numpy().reshape(-1,1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df['kudo_diffs_avg_scaled'] = x_scaled
df = df.sort_values(by=['kudo_diffs_avg_scaled'])

df.tail(10)

,work_id,title,rating,category,fandom,relationship,character,additional tags,language,published,status,status date,words,chapters,comments,kudos,bookmarks,hits,num_chap,pred_kudos,kudo_diffs,kudo_diffs_avg,kudo_diffs_avg_scaled
1461,3396131,Chance Meetings,General Audiences,Gen,"Star Wars: Rebels, Star Wars - All Media Types...",No Relationship Specified,"Kanan Jarrus | Caleb Dume, Sugi","Backstory, Order 66",English,2015-02-20,Completed,2015-02-20,"1,787.00",1/1,16.00,316.00,29.00,"2,116.00",1,109.67,206.33,0.65,0.99
1896,5068015,Relics,General Audiences,No Category Specified,"Star Wars Original Trilogy, Star Wars: The Clo...",No Relationship Specified,"Luke Skywalker, CT-7567 | Rex",Feelings,English,2015-10-24,Completed,2015-10-24,"1,456.00",1/1,20.00,611.00,103.00,"4,490.00",1,203.11,407.89,0.67,0.99
2405,8032450,Balance Point,General Audiences,Gen,"Star Wars - All Media Types, Star Wars: The Cl...",Anakin Skywalker & Ahsoka Tano,"Anakin Skywalker | Darth Vader, Ahsoka Tano","Force Ghosts, Post-Episode: s02e19-20 Twilight...",English,2016-09-14,Completed,2016-09-14,"2,339.00",1/1,31.00,537.00,113.00,"3,533.00",1,165.37,371.63,0.69,0.99
3659,4527786,Hide Yourself for Me,General Audiences,Gen,"Star Wars: Rebels, Star Wars Original Trilogy",Ahsoka Tano & Leia Organa,"Ahsoka Tano, Leia Organa, Bail Organa",No Additional Tags Specified,English,2015-08-08,Completed,2015-08-08,"2,209.00",1/1,28.00,475.00,101.00,"2,904.00",1,140.64,334.36,0.70,0.99
1000,19760368,The Weird Encounter,Mature,F/M,Star Wars: Rebels,"Ezra Bridger/Seventh Sister, Ezra Bridger/Hera...","Ezra Bridger, Kanan Jarrus, Seventh Sister, C1...","Coming back to the light, Space Exploration, R...",English,2019-07-11,Updated,2019-07-31,"2,810.00",8/?,7.00,181.00,0.00,683.00,8,48.34,132.66,0.73,0.99
3048,17348450,"Exit, Pursued by a... what in wild space is th...",Mature,"F/F, F/M, M/M, Multi","Star Wars: Rebels, Star Wars - All Media Types",Thrawn | Mitth'raw'nuruodo/Original Character(...,"Thrawn | Mitth'raw'nuruodo, Eli Vanto, Jaina B...",Not everyone in the Empire was evil and I thin...,English,2019-02-06,Updated,2019-04-07,"173,993.00",33/?,11.00,16.00,3.00,432.00,33,4.16,11.84,0.74,0.99
2017,19401877,The Stewardess,Mature,F/M,Star Wars: Thrawn Series - Timothy Zahn (2017)...,Thrawn | Mitth'raw'nuruodo/Original Female Cha...,"Thrawn | Mitth'raw'nuruodo, Original Female Ch...","Slow Burn, Angst, Fluff, Friends to Lovers, Ma...",English,2019-06-29,Updated,2019-12-28,"295,743.00",55/?,148.00,163.00,14.00,"1,766.00",55,29.32,133.68,0.82,0.99
1950,17218841,See Me,Mature,F/M,"Star Wars - All Media Types, Star Wars: Rebels...",Thrawn | Mitth'raw'nuruodo/Original Female Cha...,"Gilad Pellaeon, Thrawn | Mitth'raw'nuruodo, Ez...","Original Character(s), original Imperial chara...",English,2018-12-30,Completed,2019-03-26,"245,834.00",66/66,101.00,67.00,6.00,"1,256.00",66,6.58,60.42,0.90,0.99
2994,16626617,Shining Future,Teen And Up Audiences,"F/F, F/M, Gen","Star Wars - All Media Types, Star Wars: The Cl...","Barriss Offee/Ahsoka Tano, Obi-Wan Kenobi/Lumi...","Barriss Offee, Luminara Unduli, Ahsoka Tano, O...","Family Feels, Psychological Trauma, Adopted Ch...",English,2018-12-02,Completed,2019-08-18,"336,397.00",111/111,"1,064.00",121.00,1.00,"2,232.00",111,4.81,116.19,0.96,0.99
2850,14209392,Iskusstvo chissov / The Art of Chiss,Explicit,F/M,"Star Wars - All Media Types, Star Wars Legends...",Maris Ferasi/Thrawn | Mitth'raw'nuruodo,"Thrass | Mitth'ras'safis, Ar'alani (Star Wars)...","Shibari, Drama & Romance, Angst and Hurt/Comfo...",Russkii,2018-04-04,Completed,2018-05-11,"75,216.00",42/42,2.00,12.00,3.00,181.00,42,-2.16,14.16,1.18,1.00


In [71]:
a = df.loc[df['work_id']==11997396]
b = df.loc[df['work_id']==6591574]
c = df.loc[df['work_id']==8225590]
d = df.loc[df['work_id']==8382031]
f = df.loc[df['work_id']==8443744]
e = df.loc[df['work_id']==9082702]
g = df.loc[df['work_id']==11081991]
h = df.loc[df['work_id']==11265918]
i = df.loc[df['work_id']==14084865]
i = df.loc[df['work_id']==11018649]
df_test = pd.concat([a,b, c, d, e, f, g, h, i], axis=0).sort_values(by=['kudo_diffs_avg'])
df_test

,work_id,title,rating,category,fandom,relationship,character,additional tags,language,published,status,status date,words,chapters,comments,kudos,bookmarks,hits,num_chap,pred_kudos,kudo_diffs,kudo_diffs_avg,kudo_diffs_avg_scaled
1677,8443744,Ardor,Explicit,F/M,"Star Wars: Rebels, Star Wars: A New Dawn - Joh...",Kanan Jarrus/Hera Syndulla,"Kanan Jarrus, Hera Syndulla","Pre-Star Wars: Rebels, Public Displays, Dry Hu...",English,2016-11-02,Completed,2016-11-16,"10,739.00",3/3,32.00,130.00,15.00,"3,180.00",3,149.26,-19.26,-0.15,0.96
2880,11265918,Nocturnal Wanderings,Mature,F/M,Star Wars: Rebels,Kanan Jarrus/Hera Syndulla,"Hera Syndulla, Kanan Jarrus","sleeping, Touching, promises of something more...",English,2017-06-21,Completed,2017-06-21,673.00,1/1,13.00,50.00,6.00,746.00,1,55.88,-5.88,-0.12,0.96
644,11018649,Hot and Cold,Explicit,F/M,"Star Wars: Rebels, Star Wars: Rebellion Era - ...",Kanan Jarrus/Hera Syndulla,"Hera Syndulla, Kanan Jarrus","kanera2k17, Huddling For Warmth, Unresolved Se...",English,2017-05-28,Completed,2017-05-28,"3,614.00",1/1,7.00,74.00,5.00,"1,362.00",1,79.82,-5.82,-0.08,0.96
2906,8225590,Susurrations,Explicit,F/M,"Star Wars: Rebels, Star Wars: A New Dawn - Joh...",Kanan Jarrus/Hera Syndulla,"Hera Syndulla, Kanan Jarrus","post-A New Dawn, UST, Masturbation, Porn, Pre-...",English,2016-10-06,Completed,2016-10-06,"5,401.00",1/1,18.00,104.00,17.00,"2,122.00",1,109.55,-5.55,-0.05,0.96
1560,9082702,Wherever You Go,Explicit,F/M,"Star Wars: Rebels, Star Wars: A New Dawn - Joh...",Kanan Jarrus/Hera Syndulla,"Hera Syndulla, Kanan Jarrus, Kleeve (Star Wars...","post-AND, Pre-Rebels, establishing a relations...",English,2016-12-28,Completed,2017-05-22,"62,009.00",12/12,157.00,199.00,23.00,"4,262.00",12,180.51,18.49,0.09,0.97
548,11997396,The Recruiter,Teen And Up Audiences,Gen,"Star Wars - All Media Types, Star Wars: Rebels",No Relationship Specified,"Kanan Jarrus, Original Characters","Jedi recruitment, Implied/Referenced Domestic ...",English,2017-09-04,Completed,2017-09-04,"5,379.00",1/1,7.00,39.00,6.00,216.00,1,34.56,4.44,0.11,0.97
420,11081991,Ten Screws,General Audiences,Gen,Star Wars: Rebels,"C1-10P | Chopper & Hera Syndulla, Kanan Jarrus...","Hera Syndulla, Kanan Jarrus","Minor Kanan Jarrus/Hera Syndulla, but it's not...",English,2017-06-03,Completed,2017-06-03,"3,894.00",1/1,25.00,66.00,10.00,525.00,1,46.86,19.14,0.29,0.97
4211,8382031,Sparring,General Audiences,Gen,Star Wars: Rebels,No Relationship Specified,"Kanan Jarrus, Ezra Bridger, Sabine Wren, Garaz...","Sparring, Master & Padawan Relationship(s), Bl...",English,2016-10-26,Completed,2016-10-26,"3,580.00",1/1,21.00,95.00,13.00,"1,047.00",1,67.43,27.57,0.29,0.97
1603,6591574,Cover of Night,Mature,F/M,"Star Wars: Rebels, Star Wars: A New Dawn - Joh...",Kanan Jarrus/Hera Syndulla,"Kanan Jarrus, Hera Syndulla","Hurt/Comfort, Dreams and Nightmares, Angst, po...",English,2016-04-18,Completed,2016-04-18,"2,383.00",1/1,18.00,127.00,11.00,"1,429.00",1,82.58,44.42,0.35,0.98


### I'm curious to find some under-rated fics. Ones that score higher than expected but have low hits.

In [88]:
above_avg = df[df['kudo_diffs_avg'] > 0]
multi = above_avg[above_avg['num_chap'] > 0]
no_pod = multi[~multi["title"].str.contains('podfic')]
unpop = no_pod[no_pod['hits']<505]
undiscovered = unpop[unpop['kudo_diffs_avg_scaled']>0.8]
print(len(undiscovered.sort_values(by=['kudo_diffs_avg_scaled']).index))
undiscovered.sort_values(by=['rating'])
#undiscovered['kudo_diffs_avg_scaled'].hist(bins=10)

200


,work_id,title,rating,category,fandom,relationship,character,additional tags,language,published,status,status date,words,chapters,comments,kudos,bookmarks,hits,num_chap,pred_kudos,kudo_diffs,kudo_diffs_avg,kudo_diffs_avg_scaled
2850,14209392,Iskusstvo chissov / The Art of Chiss,Explicit,F/M,"Star Wars - All Media Types, Star Wars Legends...",Maris Ferasi/Thrawn | Mitth'raw'nuruodo,"Thrass | Mitth'ras'safis, Ar'alani (Star Wars)...","Shibari, Drama & Romance, Angst and Hurt/Comfo...",Russkii,2018-04-04,Completed,2018-05-11,"75,216.00",42/42,2.00,12.00,3.00,181.00,42,-2.16,14.16,1.18,1.00
365,14067336,Imperskie drabbly,Explicit,"Gen, M/M, Multi","Star Wars - All Media Types, Star Wars: Rebels","Gilad Pellaeon/Thrawn | Mitth'raw'nuruodo, Thr...","Gilad Pellaeon, Thrawn | Mitth'raw'nuruodo, Ma...","Lace Panties, thigh highs, Cuddling, Fluff, Sh...",Russkii,2018-03-23,Updated,2019-06-15,"38,545.00",37/?,1.00,15.00,0.00,291.00,37,9.27,5.73,0.38,0.98
2731,15213872,Thryce On Ice!,Explicit,F/M,"Star Wars: Rebels, Star Wars: Thrawn - Timothy...","Arihnda Pryce/Thrawn | Mitth'raw'nuruodo, thry...","Thrawn | Mitth'raw'nuruodo, Arihnda Pryce, Jua...","figure skating AU, yuri on ice rip-off, cameos...",English,2018-07-08,Updated,2019-04-14,"45,410.00",10/13,77.00,47.00,5.00,470.00,10,34.35,12.65,0.27,0.97
751,13661856,Silence Reigns,General Audiences,Gen,"Star Wars - All Media Types, Star Wars: Rebels","Kanan Jarrus/Hera Syndulla, Ezra Bridger & Kan...","Hera Syndulla, Ezra Bridger","Mourning, What-If, this is a very sad fic",English,2018-02-13,Completed,2018-02-13,"1,109.00",1/1,7.00,53.00,5.00,467.00,1,44.86,8.14,0.15,0.97
4139,14085450,Fortune,General Audiences,No Category Specified,Star Wars: Rebels,Janus Kasmir & Caleb Dume,"Janus Kasmir, Caleb Dume","Alien April, JediFest",English,2018-03-25,Completed,2018-03-25,770.00,1/1,8.00,43.00,2.00,251.00,1,36.39,6.61,0.15,0.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2744,20179681,The Way Home,Teen And Up Audiences,M/M,"Star Wars - All Media Types, Star Wars: Rebels...","Thrawn | Mitth'raw'nuruodo/Eli Vanto, Karyn Fa...","Eli Vanto, Thrawn | Mitth'raw'nuruodo, Karyn F...",Thranto,English,2019-08-09,Completed,2019-08-09,890.00,1/1,8.00,44.00,2.00,394.00,1,42.01,1.99,0.05,0.97
4166,13767177,Gone,Teen And Up Audiences,"F/M, Gen",Star Wars: Rebels,"Kanan Jarrus/Hera Syndulla, Kanan Jarrus & Her...","Hera Syndulla, Kanan Jarrus","Post-Jedi Night, Not Entirely Real, Or Possibl...",English,2018-02-22,Completed,2018-02-22,"1,353.00",1/1,5.00,42.00,3.00,346.00,1,40.07,1.93,0.05,0.97
100,21496054,Chopper vs. the Angry Soulmate Goose,Teen And Up Audiences,F/M,"Star Wars - All Media Types, Star Wars: Rebels...",Kanan Jarrus/Hera Syndulla,"Hera Syndulla, Chopper (Star Wars), Kanan Jarr...","The Soulmate Goose, Fix-It, Notfic, Goose-typi...",English,2019-11-20,Completed,2019-11-20,"1,704.00",1/1,17.00,49.00,4.00,306.00,1,38.46,10.54,0.22,0.97
3008,14212965,Seeds of Rebellion,Teen And Up Audiences,Gen,"Star Wars: Rebels, Star Wars: Rise of Empire E...","Alexsandr Kallus & Garazeb ""Zeb"" Orrelios","Alexsandr Kallus, Garazeb ""Zeb"" Orrelios","Angst, Introspection, Post-Episode: s02e17 The...",English,2018-04-04,Completed,2018-04-04,"2,627.00",1/1,14.00,58.00,6.00,423.00,1,42.98,15.02,0.26,0.97


### Alternately how about we look for low hits based on how long it's been out

In [103]:
import datetime
pd.options.display.float_format = '{:,.4f}'.format

scrape_date = datetime.datetime(2019, 12, 27)
undiscovered['days'] = scrape_date - pd.to_datetime(undiscovered['status date'])
undiscovered
#undiscovered['hits']
undiscovered['hits_per_day'] = undiscovered['hits']/undiscovered['days'].astype(int)
undiscovered.sort_values(by=['hits_per_day'])

,work_id,title,rating,category,fandom,relationship,character,additional tags,language,published,status,status date,words,chapters,comments,kudos,bookmarks,hits,num_chap,pred_kudos,kudo_diffs,kudo_diffs_avg,kudo_diffs_avg_scaled,days,hits_per_day
1337,10557828,Stolen Idols,General Audiences,No Category Specified,Star Wars: Rebels,No Relationship Specified,Thrawn | Mitth'raw'nuruodo,No Additional Tags Specified,English,2017-04-07,Completed,2017-04-07,"1,530.0000",1/1,3.0000,38.0000,2.0000,231.0000,1,35.5312,2.4688,0.0650,0.9683,994 days,0.0000
4199,10003226,What We Need Is a Distraction,General Audiences,Gen,"Star Wars - All Media Types, Star Wars: Rebels...",No Relationship Specified,"Sabine Wren, Fenn Rau","POV Female Character, Missing Scene, Wakes & F...",English,2017-02-27,Completed,2017-02-27,"3,325.0000",1/1,4.0000,42.0000,0.0000,241.0000,1,35.7467,6.2533,0.1489,0.9707,1033 days,0.0000
548,11997396,The Recruiter,Teen And Up Audiences,Gen,"Star Wars - All Media Types, Star Wars: Rebels",No Relationship Specified,"Kanan Jarrus, Original Characters","Jedi recruitment, Implied/Referenced Domestic ...",English,2017-09-04,Completed,2017-09-04,"5,379.0000",1/1,7.0000,39.0000,6.0000,216.0000,1,34.5593,4.4407,0.1139,0.9697,844 days,0.0000
3086,3792697,Human Boys,General Audiences,Gen,"Star Wars - All Media Types, Star Wars: The Cl...",No Relationship Specified,"Hera Syndulla, Kanan Jarrus, Depa Billaba, Num...",No Additional Tags Specified,English,2015-01-17,Completed,2015-01-17,"2,637.0000",1/1,1.0000,48.0000,3.0000,473.0000,1,44.9431,3.0569,0.0637,0.9683,1805 days,0.0000
2802,7637077,Stargazers,General Audiences,Gen,Star Wars: Rebels,No Relationship Specified,"Hera Syndulla, Kanan Jarrus, Garazeb ""Zeb"" Orr...",No Additional Tags Specified,English,2016-07-31,Completed,2016-07-31,"1,409.0000",1/1,0.0000,40.0000,7.0000,335.0000,1,39.6352,0.3648,0.0091,0.9667,1244 days,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3167,21640723,Circling the Wagons,Teen And Up Audiences,Gen,Star Wars: Rebels,"Ezra Bridger & The Ghost Crew, Ezra Bridger & ...","The Ghost Crew (Star Wars), Ezra Bridger, Kana...","Found Family, Protectiveness, some creepy OCs,...",English,2019-12-02,Completed,2019-12-01,"14,782.0000",7/7,4.0000,56.0000,17.0000,473.0000,7,39.5802,16.4198,0.2932,0.9748,26 days,0.0000
3302,17504063,Rebels Tarot Deck,General Audiences,Gen,"Star Wars - All Media Types, Star Wars: Rebels",Kanan Jarrus/Hera Syndulla,"Ahsoka Tano, Ezra Bridger, Kanan Jarrus, Hera ...","Tarot, major arcana - Freeform, Character Arch...",English,2019-12-04,Updated,2019-12-04,75.0000,7/22,6.0000,44.0000,2.0000,485.0000,7,41.5112,2.4888,0.0566,0.9681,23 days,0.0000
3045,21862429,Interspecies Relations,Teen And Up Audiences,M/M,Star Wars: Rebels,"Alexsandr Kallus/Garazeb ""Zeb"" Orrelios",Alexsandr Kallus,"Fluff, Rumors",English,2019-12-19,Completed,2019-12-19,"1,585.0000",1/1,6.0000,54.0000,2.0000,325.0000,1,39.2243,14.7757,0.2736,0.9742,8 days,0.0000
3041,21907324,Pistachio,General Audiences,"Gen, M/M","Star Wars - All Media Types, Star Wars Sequel ...","Alexsandr Kallus/Garazeb ""Zeb"" Orrelios, Armit...","Armitage Hux, Rae Sloane, Alexsandr Kallus, Ga...","Rise of Skywalker Spoilers, A happy ending for...",English,2019-12-22,Completed,2019-12-22,"1,908.0000",1/1,16.0000,47.0000,7.0000,288.0000,1,37.7365,9.2635,0.1971,0.9721,5 days,0.0000
